In [1]:
import os
import sys
import keras_tuner
from dotenv import load_dotenv

from usd_idr_forecasting.utils import get_dt_now
from usd_idr_forecasting.configs import ProjectConfig
from usd_idr_forecasting.tuner import Tuner
from usd_idr_forecasting.models import TemporalHyperModel
from usd_idr_forecasting.data.datasets import DatasetLoader

load_dotenv()
PROJECT_WORKING_DIR = os.getenv("PROJECT_WORKING_DIR")

/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/heykalsayid/.netrc
wandb: Currently logged in as: heykalsayid (danielteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# **Load Config**

In [2]:

config_path = os.path.join(PROJECT_WORKING_DIR, 'project_configs.yaml')
project_config = ProjectConfig.from_yaml(config_path)
project_config

ProjectConfig(project_name='idrx-forecast', wandb_team_name='danielteam', general={'windowing_size': 42, 'target_size': 1, 'batch_size': [8, 16, 32], 'shuffle_buffer_size': None}, model={'learning_rate': 0.0005, 'momentum': 0.9, 'optimizer': ['adam'], 'epochs': 50, 'rnn': {'units': 128, 'return_sequences': True}, 'conv1d': {'filter': 32, 'kernel_size': 3, 'strides': 1, 'padding': 'causal', 'activation': 'relu', 'name': 'conv1d'}, 'max_pooling_1d': 42, 'dropout': 0.2, 'feed_forward_layer': {'dense_1': {'dense': 10, 'activation': 'relu', 'name': 'dense_1'}, 'output': {'dense': 1, 'name': 'output'}}}, tuner={'batch_size': 32, 'rnn_units': {'min_value': 50, 'max_value': 200, 'step': 30}, 'conv1d': {'filter': {'min_value': 50, 'max_value': 100, 'step': 25}, 'kernel_size': [3, 42]}, 'max_pool1d': {'name': 'max_pool1d'}, 'flattening_layer': ['GlobalAveragePooling1D', 'GlobalMaxPool1D'], 'batch_norm': {'name': 'batch_norm'}, 'dropout': {'name': 'dropout', 'rate': [0.25, 0.2]}, 'dense_units': [

# **Load Datasets**

In [3]:
train_set, valid_set, batch_size, prep_artifact = DatasetLoader(config=project_config).load_dataset_for_training(batch_size=32)

wandb:   37 of 37 files downloaded.  


available preprocessed datasets:
	['train@preprocessed-ds_2025121_172934_626424_batch@8', 'scaler.pkl', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid@preprocessed-ds_2025121_172934_626424_batch@8', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@16', 'valid@preprocessed-ds_2025121_172934_626424_batch@16', 'train@preprocessed-ds_2025121_172934_626424_batch@32', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid@preprocessed-ds_2025121_172934_626424_batch@32', 'train@preprocessed-ds_2025121_172934_626424_batch@16', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@16']


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


['train@preprocessed-ds_2025121_172934_626424_batch@8', 'scaler.pkl', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid@preprocessed-ds_2025121_172934_626424_batch@8', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@16', 'valid@preprocessed-ds_2025121_172934_626424_batch@16', 'train@preprocessed-ds_2025121_172934_626424_batch@32', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid@preprocessed-ds_2025121_172934_626424_batch@32', 'train@preprocessed-ds_2025121_172934_626424_batch@16', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@16']
train index: 8
valid index: 10


# **Tuning**

In [4]:
process_id = get_dt_now()
print(f'Process Tuning ID: {process_id}')

Process Tuning ID: 2026118_183657_197770


In [5]:

model_lstm_tuner = TemporalHyperModel(config=project_config, model_type='lstm')
lstm_tuner = Tuner(
	hypermodel=model_lstm_tuner,
    objective=keras_tuner.Objective('val_rmse', direction='min'),
    max_trials=2,
    executions_per_trial=1,
    overwrite=False,
    directory="lstm_tuner",
    project_name=f"tuned@lstm_{process_id}",
    config=project_config,
    model_type='lstm',
    process_id=process_id,
)


model_gru_tuner = TemporalHyperModel(config=project_config, model_type='gru')
gru_tuner = Tuner(
	hypermodel=model_gru_tuner,
    objective=keras_tuner.Objective('val_rmse', direction='min'),
    max_trials=2,
    executions_per_trial=1,
    overwrite=False,
    directory="gru_tuner",
    project_name=f"tuned@gru_{process_id}",
    config=project_config,
    model_type='gru',
    process_id=process_id,
)


In [6]:
lstm_tuner.search_space_summary()

Search space summary
Default search space size: 11
rnn_units (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 200, 'step': 30, 'sampling': 'linear'}
conv1d_filters (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 25, 'sampling': 'linear'}
conv1d_kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 42], 'ordered': True}
max_pool1d (Boolean)
{'default': False, 'conditions': []}
flattening_layer (Choice)
{'default': 'GlobalAveragePooling1D', 'conditions': [], 'values': ['GlobalAveragePooling1D', 'GlobalMaxPool1D'], 'ordered': False}
batch_norm (Boolean)
{'default': False, 'conditions': []}
dropout (Boolean)
{'default': False, 'conditions': []}
dropout_rate (Choice)
{'default': 0.25, 'conditions': [], 'values': [0.25, 0.2], 'ordered': True}
dense_units (Choice)
{'default': 50, 'conditions': [], 'values': [50, 42], 'ordered': True}
learning_rate (Choice)
{'default': 5e-05, 'conditions': [], 'values': [5e-05, 0.0005], 

In [7]:
gru_tuner.search_space_summary()

Search space summary
Default search space size: 11
rnn_units (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 200, 'step': 30, 'sampling': 'linear'}
conv1d_filters (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 25, 'sampling': 'linear'}
conv1d_kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 42], 'ordered': True}
max_pool1d (Boolean)
{'default': False, 'conditions': []}
flattening_layer (Choice)
{'default': 'GlobalAveragePooling1D', 'conditions': [], 'values': ['GlobalAveragePooling1D', 'GlobalMaxPool1D'], 'ordered': False}
batch_norm (Boolean)
{'default': False, 'conditions': []}
dropout (Boolean)
{'default': False, 'conditions': []}
dropout_rate (Choice)
{'default': 0.25, 'conditions': [], 'values': [0.25, 0.2], 'ordered': True}
dense_units (Choice)
{'default': 50, 'conditions': [], 'values': [50, 42], 'ordered': True}
learning_rate (Choice)
{'default': 5e-05, 'conditions': [], 'values': [5e-05, 0.0005], 

## **Start Tuning**

### **LSTM**

In [8]:
lstm_tuner.start(train_set, valid_set)

Trial 2 Complete [00h 01m 08s]
val_rmse: 0.6314115524291992

Best val_rmse So Far: 0.043691832572221756
Total elapsed time: 00h 01m 56s


wandb: Adding directory to artifact (/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/models/tuned/RandomSearch/lstm)... Done. 0.0s
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 25 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


[<Functional name=functional, built=True>,
 <Functional name=functional, built=True>]

### **GRU**

In [11]:
gru_tuner.start(train_set, valid_set)

Trial 2 Complete [00h 00m 25s]
val_rmse: 0.03901999816298485

Best val_rmse So Far: 0.03901999816298485
Total elapsed time: 00h 01m 09s


wandb: Adding directory to artifact (/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/models/tuned/RandomSearch/gru)... Done. 0.0s
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 21 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


[<Functional name=functional, built=True>,
 <Functional name=functional, built=True>]

In [10]:
print("This will only print to the screen once.")

This will only print to the screen once.
